## NLP Phase 3: modeling

	
In the final phase of the project, we will use the datasets created in phase 2 to build a text categorization model.

#### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import pickle
import nltk.stem as stemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from porter2stemmer import Porter2Stemmer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from textblob import TextBlob


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kriti\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kriti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kriti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Importing Primary Dataset:

In [2]:
df_prim_lock=pd.read_csv(r"C:\Users\Kriti\Desktop\College-Academics\NLP\Project Phase 2-\Project_Phase_2_kshree\Combined_Datasets_kshree\Primary_Dataset\nyt_topic_lockdowns.csv")
df_prim_m_d=pd.read_csv(r"C:\Users\Kriti\Desktop\College-Academics\NLP\Project Phase 2-\Project_Phase_2_kshree\Combined_Datasets_kshree\Primary_Dataset\nyt_topic_masking_and_distancing.csv")
df_prim_vacc=pd.read_csv(r"C:\Users\Kriti\Desktop\College-Academics\NLP\Project Phase 2-\Project_Phase_2_kshree\Combined_Datasets_kshree\Primary_Dataset\nyt_topic_vaccination.csv")

In [3]:
df_second_lock=pd.read_csv(r"C:\Users\Kriti\Desktop\College-Academics\NLP\Project Phase 2-\Project_Phase_2_kshree\Combined_Datasets_kshree\Secondary_Dataset\change.org_topic_lockdowns.csv")
df_second_m_d=pd.read_csv(r"C:\Users\Kriti\Desktop\College-Academics\NLP\Project Phase 2-\Project_Phase_2_kshree\Combined_Datasets_kshree\Secondary_Dataset\change.org_topic_masking_and_distancing.csv")
df_second_vacc=pd.read_csv(r"C:\Users\Kriti\Desktop\College-Academics\NLP\Project Phase 2-\Project_Phase_2_kshree\Combined_Datasets_kshree\Secondary_Dataset\change.org_topic_vaccination.csv")

In [4]:
df_prim_lock.head()

text  final_label
0  What do you mean that the pandemic has stretch...        False
1  I’m not sure if it qualifies as Big Brother wh...        False
2  What if smallpox and polio came roaring back o...        False
3  I'm continuing to wear a mask.  Even in bed, s...        False
4  Although my husband and I really, really miss ...        False

In [5]:
df_prim_lock.describe()

text final_label
count                                                 813         813
unique                                                808           2
top     What about people who have already had the vir...       False
freq                                                    3         711

In [6]:
df_prim_m_d.head()

text  final_label
0  What do you mean that the pandemic has stretch...        False
1  I’m not sure if it qualifies as Big Brother wh...        False
2  What if smallpox and polio came roaring back o...        False
3  I'm continuing to wear a mask.  Even in bed, s...         True
4  Although my husband and I really, really miss ...        False

In [7]:
df_prim_m_d.describe()

text final_label
count                                                 813         813
unique                                                808           2
top     What about people who have already had the vir...       False
freq                                                    3         586

In [8]:
df_prim_vacc.head()

text  final_label
0  What do you mean that the pandemic has stretch...        False
1  I’m not sure if it qualifies as Big Brother wh...        False
2  What if smallpox and polio came roaring back o...         True
3  I'm continuing to wear a mask.  Even in bed, s...         True
4  Although my husband and I really, really miss ...        False

In [9]:
df_prim_vacc.describe()

text final_label
count                                                 813         813
unique                                                808           2
top     What about people who have already had the vir...        True
freq                                                    3         496

#### To select dataset for modeling we will calculate positive class. We will avoid topics with low number of positive classes

In [10]:
print("\nRatio of Classes for NYT LOCKDOWN\n",df_prim_lock.final_label.value_counts(True))
print("\nRatio of Classes for NYT MASKING AND DISTANCING\n",df_prim_m_d.final_label.value_counts(True))
print("\nRatio of Classes for NYT VACCINATION\n",df_prim_vacc.final_label.value_counts(True))


Ratio of Classes for NYT LOCKDOWN
 False    0.874539
True     0.125461
Name: final_label, dtype: float64

Ratio of Classes for NYT MASKING AND DISTANCING
 False    0.720787
True     0.279213
Name: final_label, dtype: float64

Ratio of Classes for NYT VACCINATION
 True     0.610086
False    0.389914
Name: final_label, dtype: float64


#### From above we can see that NYT Vaccination has high number of positive classes as compared to NYT Lockdown and NYT Masking and Distancing. We will take NYT Vaccination dataset to build a text categorization model.

<strong>1. Data partitioning:</strong> In this part we will create training set for the model by randomly selecting 70% of the texts in NYT Vaccination. Remaning 30% is the Test set. 
Data Partitioning can be done by using various methods. In this project we have used <strong>train_test_split</strong> to split dataset in Train and Test Dataset 



In [11]:
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(df_prim_vacc.text, df_prim_vacc.final_label, test_size=0.3,random_state=11)

print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(569,) (244,) (569,) (244,)


In [12]:
#Training data 
training_data_f = pd.concat([X_train,y_train],axis=1)
training_data_f.head()

text  final_label
541  I DON'T UNDERSTAND! the hesitancy of employers...         True
155  Every day I become angrier and angrier about t...         True
127  I am tired of hearing about "delicate balances...         True
801  In the USA we are already divided by wealth, r...         True
410  “Don’t Tread on Me”, one of the Trump Insurrec...         True

In [13]:
#Test Data(Primary)
test_data_f = pd.concat([X_test,y_test],axis=1)
test_data_f.head()


text  final_label
183  This whole thing is so stupid. Stop already wi...        False
123  Imagine if Covid killed 50% of those who came ...         True
25   I have a close friend who is refusing to be va...         True
192  Not sure how the personal privacy fits in the ...         True
46   There is no delicate balance. Private firms or...         True

### Baseline Modeling: Train a simple bag-of-words classifier on Training dataset(training_data_f).

The bag of words model converts text into numerical representations (numerical feature vectors), which may then be used to train machine learning algorithms. The following are the main processes in putting together a bag-of-words model:
1. Using the whole collection of texts, create a vocabulary indices of words or tokens. The alphabetical arrangement of the vocabulary indices is a possibility.
2. Create a numerical feature vector for each document that indicates the frequency with which each word appears in various documents.

#### Converting train and test data into vectors

In [14]:
#Converting train data into vectors
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
matrix = TfidfVectorizer(max_features=1000)
data_f_train = matrix.fit_transform(training_data_f['text'],training_data_f['final_label'])

print(data_f_train.shape)
print(sorted(matrix.vocabulary_))

(569, 1000)
['000', '10', '100', '12', '19', '20', '40', '50', '80', 'able', 'about', 'absolutely', 'accept', 'access', 'across', 'act', 'actions', 'actually', 'add', 'adequate', 'administration', 'adults', 'after', 'again', 'against', 'age', 'ago', 'agree', 'air', 'airline', 'all', 'allow', 'allowed', 'almost', 'alone', 'already', 'also', 'alternative', 'although', 'always', 'am', 'america', 'american', 'americans', 'among', 'amount', 'an', 'and', 'another', 'answer', 'anti', 'antibodies', 'antibody', 'any', 'anyone', 'anything', 'anyway', 'anywhere', 'approach', 'approval', 'approved', 'are', 'area', 'areas', 'aren', 'around', 'article', 'as', 'ask', 'asked', 'associated', 'asymptomatic', 'at', 'authorization', 'available', 'avoid', 'away', 'back', 'bad', 'bar', 'bars', 'based', 'basis', 'be', 'because', 'become', 'been', 'before', 'begin', 'beginning', 'behavior', 'behind', 'being', 'beings', 'believe', 'benefit', 'benefits', 'best', 'better', 'between', 'biden', 'big', 'bit', 'blam

In [15]:
#Converting test data into vectors
matrix = TfidfVectorizer(max_features=1000)
# data_f = matrix.fit_transform(train_prim['text'],train_prim['Final Label'])
data_f_test = matrix.fit_transform(test_data_f['text'],test_data_f['final_label'])
print(data_f_test.shape)

(244, 1000)


#### Training GaussianNB model- Primary Dataset

In [16]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(data_f_train.toarray(), y_train)

GaussianNB()

#### Baseline model's overall accuracy for the Test (PRIMARY) 

In [17]:
# predict class
y_pred_1 = classifier.predict(data_f_test.toarray())

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm1 = confusion_matrix(y_test, y_pred_1)
cr1 = classification_report(y_test, y_pred_1)
accuracy1 = accuracy_score(y_test, y_pred_1)
print("Confusion Matrix\n",cm1)
print("Classification Report\n", cr1)
print("Accuracy Score\n", accuracy1)

Confusion Matrix
 [[  4  82]
 [  7 151]]
Classification Report
               precision    recall  f1-score   support

       False       0.36      0.05      0.08        86
        True       0.65      0.96      0.77       158

    accuracy                           0.64       244
   macro avg       0.51      0.50      0.43       244
weighted avg       0.55      0.64      0.53       244

Accuracy Score
 0.6352459016393442


#### Now we will look into Secondary Dataset and do all the above procedures on it

In [18]:
df_second_lock.head()

text  final_label
0         Language Education in the Time of COVID-19        False
1                                 COVID-19 Test Kits        False
2                                 COVID 19 IN PRISON        False
3                                     Get Waled Home        False
4  Make pass/fail available for Mississippi State...        False

In [19]:
df_second_lock.describe()

text final_label
count                                                1500        1500
unique                                               1500           2
top     Humanitarian Appeal to Release Maulana Delawar...       False
freq                                                    1        1071

In [20]:
df_second_m_d.head()

text  final_label
0         Language Education in the Time of COVID-19        False
1                                 COVID-19 Test Kits        False
2                                 COVID 19 IN PRISON        False
3                                     Get Waled Home        False
4  Make pass/fail available for Mississippi State...        False

In [21]:
df_second_m_d.describe()

text final_label
count                                                1500        1500
unique                                               1500           2
top     Humanitarian Appeal to Release Maulana Delawar...       False
freq                                                    1        1419

In [22]:
df_second_vacc.head()

text  final_label
0         Language Education in the Time of COVID-19        False
1                                 COVID-19 Test Kits        False
2                                 COVID 19 IN PRISON        False
3                                     Get Waled Home        False
4  Make pass/fail available for Mississippi State...        False

In [23]:
df_second_vacc.describe()

text final_label
count                                                1500        1500
unique                                               1500           2
top     Humanitarian Appeal to Release Maulana Delawar...       False
freq                                                    1        1472

In [24]:
print("\nRatio of Classes for Change.org LOCKDOWN\n",df_second_lock.final_label.value_counts(True))
print("\nRatio of Classes for Change.org MASKING AND DISTANCING\n",df_second_m_d.final_label.value_counts(True))
print("\nRatio of Classes for Change.org VACCINATION\n",df_second_vacc.final_label.value_counts(True))


Ratio of Classes for Change.org LOCKDOWN
 False    0.714
True     0.286
Name: final_label, dtype: float64

Ratio of Classes for Change.org MASKING AND DISTANCING
 False    0.946
True     0.054
Name: final_label, dtype: float64

Ratio of Classes for Change.org VACCINATION
 False    0.981333
True     0.018667
Name: final_label, dtype: float64


#### We will go ahead with Change.org Vaccination to build model. 
As per guidelines, we will be designating 100% of Dataset as Testing Dataset. For modeling, we will be using Training dataset of Primary and Testing dataset of Secondary Dataset
1. Training dataset = Primary Data 70%
2. Test Dataset = Secondary Data 100%

In [25]:
test_data_second=df_second_vacc.copy()

#### Converting Test Data(Secondary Dataset) into vectors

In [26]:
matrix = TfidfVectorizer(max_features=1000)
# data_f = matrix.fit_transform(train_prim['text'],train_prim['Final Label'])
data_second_test = matrix.fit_transform(test_data_second['text'],test_data_second['final_label'])
print(data_second_test.shape)
print(sorted(matrix.vocabulary_))

(1500, 1000)
['000', '10', '12', '15', '19', '20', '2019', '2020', '2021', '25', '30', '4th', '50', 'abbott', 'academic', 'access', 'accessible', 'accountable', 'accounts', 'across', 'act', 'action', 'activities', 'added', 'adult', 'advisory', 'affected', 'affordable', 'africa', 'african', 'after', 'again', 'against', 'agency', 'agents', 'agli', 'aid', 'air', 'airlines', 'al', 'alberta', 'all', 'allow', 'allowing', 'amend', 'american', 'americans', 'amid', 'amidst', 'amount', 'an', 'and', 'animal', 'anti', 'appeal', 'approval', 'are', 'arts', 'as', 'asap', 'asian', 'ask', 'assessment', 'assistance', 'at', 'athletes', 'attacks', 'austin', 'australia', 'australian', 'available', 'avoid', 'award', 'awareness', 'away', 'baby', 'back', 'baker', 'ban', 'banks', 'based', 'basic', 'battling', 'bc', 'be', 'beach', 'beaches', 'beauty', 'beca', 'because', 'before', 'behind', 'being', 'benefit', 'bengaluru', 'better', 'beyond', 'biased', 'biden', 'bihar', 'bill', 'bills', 'bin', 'bio', 'black', 'b

#### Training Gaussian Model

In [27]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(data_f_train.toarray(), y_train)

GaussianNB()

#### Baseline model's overall accuracy for the Test (Secondary) 

In [28]:
# predict class
y_pred_second = classifier.predict(data_second_test.toarray())

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm_second = confusion_matrix(test_data_second['final_label'], y_pred_second)
cr_second = classification_report(test_data_second['final_label'], y_pred_second)
accuracy_secondary = accuracy_score(test_data_second['final_label'], y_pred_second)
print("Confusion Matrix\n",cm_second)
print("Classification Report\n", cr_second)
print("Accuracy\n", accuracy_secondary)

Confusion Matrix
 [[666 806]
 [ 14  14]]
Classification Report
               precision    recall  f1-score   support

       False       0.98      0.45      0.62      1472
        True       0.02      0.50      0.03        28

    accuracy                           0.45      1500
   macro avg       0.50      0.48      0.33      1500
weighted avg       0.96      0.45      0.61      1500

Accuracy
 0.4533333333333333


## Model Evaluation 1:

From the above model built for Primary Secondary Dataset following can be noted down:
1. Primary Dataset: Training Dataset is 70% and Test Dataset is 30%
- <strong>  Baseline model's overall accuracy for the Test (PRIMARY)~ 64%</strong>
2. Secondary Dataset: Training Dataset is 70% Primary and Test Dataset is 100% of Secondary Dataset.
- <strong>  Baseline model's overall accuracy for the Test (SECONDARY)~ 45%</strong>

## Feature Engineering:

#### 1. Number of Characters
Count the number of characters present 

In [29]:
def count_chars(text):
    return len(text)

#### 2. Number of words
Count the number of words present in a text.

In [30]:
def count_words(text):
    return len(text.split())

#### 3. Number of capital characters
Count the number of capital characters present in a text.

In [31]:
def count_capital_chars(text):
    count=0
    for i in text:
        if i.isupper():
            count+=1
    return count

#### 4. Number of capital words
Count the number of capital words present in a text.

In [32]:
def count_capital_words(text):
    return sum(map(str.isupper,text.split()))

#### 5. Count the number of punctuations
In this function, we return a dictionary of 32 punctuation with the counts

In [33]:
def count_punctuations(text):
    punctuations='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'
    d=dict()
    for i in punctuations:
        d[str(i)+' count']=text.count(i)
    return d 

#### 6. Number of words in quotes
The number of words in the single quotation and double quotation. 

In [34]:
def count_words_in_quotes(text):
    x = re.findall("'.'| ", text)
    count=0
    if x is None:
        return 0
    else:
        for i in x:
            t=i[1:-1]
            count+=count_words(t)
        return count

#### 7. Count the number of unique words
Count the number of unique words in a text.

In [35]:
def count_unique_words(text):
    return len(set(text.split()))

#### 8. Count of stopwords
Here we will count the number of stopwords used in a text.

In [36]:
def count_stopwords(text):
    stop_words = set(stopwords.words('english'))  
    word_tokens = word_tokenize(text)
    stopwords_x = [w for w in word_tokens if w in stop_words]
    return len(stopwords_x)

In [37]:
def count_sent(text):
    return len(nltk.sent_tokenize(text))

def count_unique_words(text):
    return len(set(text.split()))


def cleaning_pipe(sent):
    
    #Removing Stopwords and Punctuations
    data = re.sub('[^a-zA-Z]', ' ',sent)
    data = data.lower()
    data = data.split()
    data = [word for word in data if not word in set(stopwords.words('english'))]
    data = ' '.join(data)
    
    #Performing Stemming using PortarStemmer2
    stemmer = Porter2Stemmer()
    words = word_tokenize(data)
    words = [ stemmer.stem(word) for word in words ]
    data = ' '.join(words)
    
    return data
    

###  Implementation on Primary Dataset

In [38]:
df_prim_vacc['char_count'] = df_prim_vacc["text"].apply(lambda x:count_chars(x))
df_prim_vacc['word_count'] = df_prim_vacc["text"].apply(lambda x:count_words(x))
df_prim_vacc['sent_count'] = df_prim_vacc["text"].apply(lambda x:count_sent(x))
df_prim_vacc['capital_char_count'] = df_prim_vacc["text"].apply(lambda x:count_capital_chars(x))
df_prim_vacc['capital_word_count'] = df_prim_vacc["text"].apply(lambda x:count_capital_words(x))

df_prim_vacc['stopword_count'] = df_prim_vacc["text"].apply(lambda x:count_stopwords(x))


df_prim_vacc['punct_count'] = df_prim_vacc["text"].apply(lambda x:count_punctuations(x))
df_prim_vacc['avg_wordlength'] = df_prim_vacc['char_count']/df_prim_vacc['word_count']
df_prim_vacc['stopwords_vs_words'] = df_prim_vacc['stopword_count']/df_prim_vacc['word_count']
df_prim_vacc["sentiment"] = df_prim_vacc["text"].apply(lambda x:TextBlob(x).sentiment.polarity)
df_prim_vacc["text"] = df_prim_vacc["text"].apply(cleaning_pipe)



In [39]:
df_prim_vacc.head()

text  final_label  char_count  \
0            mean pandem stretch longer anyon expect        False          80   
1       sure qualifi big brother peopl health affect        False          90   
2  smallpox polio came roar back one day would si...         True         188   
3  continu wear mask even bed sleep keep burglar ...         True         150   
4  although husband realli realli miss walk town ...        False         272   

   word_count  sent_count  capital_char_count  capital_word_count  \
0          14           1                   1                   0   
1          17           1                   3                   0   
2          35           4                   5                   0   
3          29           3                   4                   1   
4          49           4                   9                   2   

   stopword_count                                        punct_count  \
0               7  {'! count': 0, '" count': 0, '# count': 0, '$ ...   
1              11  {'! count': 0, '" count': 0, '# count': 0, '$ ...   
2              11  {'! count': 0, '" count': 0, '# count': 0, '$ ...   
3              13  {'! count': 0, '" count': 0, '# count': 0, '$ ...   
4              20  {'! count': 0, '" count': 0, '# count': 0, '$ ...   

   avg_wordlength  stopwords_vs_words  sentiment  
0        5.714286            0.500000  -0.154167  
1        5.294118            0.647059  -0.125000  
2        5.371429            0.314286  -0.166667  
3        5.172414            0.448276   0.000000  
4        5.551020            0.408163   0.016667

In [40]:
# create training and testing vars
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(df_prim_vacc.text, df_prim_vacc.final_label, test_size=0.3,random_state=11)

print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(569,) (244,) (569,) (244,)


In [41]:
training_data_fe = pd.concat([X_train_f,y_train_f],axis=1)
training_data_fe.head()

text  final_label
541  understand hesit employ mandat covid vaccin wo...         True
155  everi day becom angri angri anti vaxxer even t...         True
127  tire hear delic balanc individu right pandem f...         True
801  usa alreadi divid wealth religion race vaccin ...         True
410  tread one trump insurrectionist favorit ralli ...         True

In [42]:
test_data_fe = pd.concat([X_test_f,y_test_f],axis=1)
test_data_fe.head()

text  final_label
183  whole thing stupid stop alreadi dehuman mask w...        False
123  imagin covid kill came contact debat mandatori...         True
25   close friend refus vaccin dialysi nurs work mu...         True
192  sure person privaci fit equat vaccin danger al...         True
46   delic balanc privat firm job requir public int...         True

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
matrix_fe = TfidfVectorizer(max_features=1000)
# data_f = matrix.fit_transform(train_prim['text'],train_prim['Final Label'])
data_fe_train = matrix_fe.fit_transform(training_data_fe['text'],training_data_fe['final_label'])
#data_f_train = matrix.fit_transform(X_train_prim,y_train_prim)

print(data_fe_train.shape)
print(sorted(matrix_fe.vocabulary_))

(569, 1000)
['abl', 'absolut', 'accept', 'access', 'across', 'act', 'action', 'activ', 'actual', 'add', 'adequ', 'administr', 'admit', 'adult', 'advanc', 'advers', 'advoc', 'affect', 'age', 'ago', 'agre', 'air', 'airlin', 'alcohol', 'allow', 'almost', 'alon', 'along', 'alreadi', 'also', 'altern', 'although', 'alway', 'amend', 'america', 'american', 'among', 'amount', 'angri', 'anoth', 'answer', 'anti', 'antibodi', 'antivaxx', 'anxieti', 'anyon', 'anyth', 'anyway', 'anywher', 'apart', 'appear', 'appli', 'approach', 'approv', 'area', 'argum', 'around', 'arriv', 'articl', 'ask', 'aspect', 'associ', 'assur', 'asymptomat', 'attend', 'author', 'avail', 'avoid', 'away', 'babi', 'back', 'bad', 'balanc', 'bar', 'base', 'basi', 'basic', 'be', 'becom', 'begin', 'behavior', 'behind', 'belief', 'believ', 'benefit', 'best', 'better', 'beyond', 'biden', 'big', 'bill', 'birth', 'bit', 'blame', 'blank', 'bodi', 'booster', 'bottom', 'brainer', 'break', 'breakthrough', 'bring', 'brother', 'build', 'busi'

In [44]:
matrix_fe = TfidfVectorizer(max_features=1000)
# data_f = matrix.fit_transform(train_prim['text'],train_prim['Final Label'])
data_fe_test = matrix_fe.fit_transform(test_data_fe['text'],test_data_fe['final_label'])
print(data_fe_test.shape)

(244, 1000)


#### Training RandomForest Model

In [45]:
#Random Forest Classifier:
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(n_estimators = 100,max_depth = 5, random_state = 10)
classifier.fit(data_fe_train.toarray(), y_train_f)

RandomForestClassifier(max_depth=5, random_state=10)

In [46]:
# predict class
y_pred_1 = classifier.predict(data_fe_test.toarray())

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm1 = confusion_matrix(y_test, y_pred_1)
cr1 = classification_report(y_test_f, y_pred_1)
accuracy1 = accuracy_score(y_test_f, y_pred_1)
print("Confusion Matrix\n",cm1)
print("Classification Report\n", cr1)
print("Accuracy\n", accuracy1)

Confusion Matrix
 [[ 77   9]
 [ 38 120]]
Classification Report
               precision    recall  f1-score   support

       False       0.67      0.90      0.77        86
        True       0.93      0.76      0.84       158

    accuracy                           0.81       244
   macro avg       0.80      0.83      0.80       244
weighted avg       0.84      0.81      0.81       244

Accuracy
 0.8073770491803278


In [47]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(data_fe_train.toarray(), y_train_f)

GaussianNB()

In [48]:
# predict class
y_pred_1 = classifier.predict(data_fe_test.toarray())

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm1 = confusion_matrix(y_test, y_pred_1)
cr1 = classification_report(y_test_f, y_pred_1)
accuracy1 = accuracy_score(y_test_f, y_pred_1)
print("Confusion Matrix\n",cm1)
print("Classification Report\n", cr1)
print("Accuracy\n", accuracy1)

Confusion Matrix
 [[ 21  65]
 [ 16 142]]
Classification Report
               precision    recall  f1-score   support

       False       0.57      0.24      0.34        86
        True       0.69      0.90      0.78       158

    accuracy                           0.67       244
   macro avg       0.63      0.57      0.56       244
weighted avg       0.64      0.67      0.62       244

Accuracy
 0.6680327868852459


### Implementation Secondary Dataset:

In [49]:
df_second_vacc['char_count'] = df_second_vacc["text"].apply(lambda x:count_chars(x))
df_second_vacc['word_count'] = df_second_vacc["text"].apply(lambda x:count_words(x))
df_second_vacc['sent_count'] = df_second_vacc["text"].apply(lambda x:count_sent(x))
df_second_vacc['capital_char_count'] = df_second_vacc["text"].apply(lambda x:count_capital_chars(x))
df_second_vacc['capital_word_count'] = df_second_vacc["text"].apply(lambda x:count_capital_words(x))

df_second_vacc['stopword_count'] = df_second_vacc["text"].apply(lambda x:count_stopwords(x))


df_second_vacc['punct_count'] = df_second_vacc["text"].apply(lambda x:count_punctuations(x))
df_second_vacc['avg_wordlength'] = df_second_vacc['char_count']/df_second_vacc['word_count']
df_second_vacc['stopwords_vs_words'] = df_second_vacc['stopword_count']/df_second_vacc['word_count']
df_second_vacc["sentiment"] = df_second_vacc["text"].apply(lambda x:TextBlob(x).sentiment.polarity)
df_second_vacc["text"] = df_second_vacc["text"].apply(cleaning_pipe)


In [50]:
df_second_vacc.isnull().sum()

text                  0
final_label           0
char_count            0
word_count            0
sent_count            0
capital_char_count    0
capital_word_count    0
stopword_count        0
punct_count           0
avg_wordlength        0
stopwords_vs_words    0
sentiment             0
dtype: int64

In [51]:
df_second_vacc.head()

text  final_label  char_count  \
0                            languag educ time covid        False          42   
1                                     covid test kit        False          18   
2                                       covid prison        False          18   
3                                      get wale home        False          14   
4  make pass fail avail mississippi state student...        False          88   

   word_count  sent_count  capital_char_count  capital_word_count  \
0           7           1                   8                   1   
1           3           1                   7                   1   
2           4           1                  13                   3   
3           3           1                   3                   0   
4          12           1                   4                   0   

   stopword_count                                        punct_count  \
0               3  {'! count': 0, '" count': 0, '# count': 0, '$ ...   
1               0  {'! count': 0, '" count': 0, '# count': 0, '$ ...   
2               0  {'! count': 0, '" count': 0, '# count': 0, '$ ...   
3               0  {'! count': 0, '" count': 0, '# count': 0, '$ ...   
4               3  {'! count': 0, '" count': 0, '# count': 0, '$ ...   

   avg_wordlength  stopwords_vs_words  sentiment  
0        6.000000            0.428571        0.0  
1        6.000000            0.000000        0.0  
2        4.500000            0.000000        0.0  
3        4.666667            0.000000        0.0  
4        7.333333            0.250000        0.2

In [52]:
test_data_second_fe=df_second_vacc.copy()

In [53]:
matrix = TfidfVectorizer(max_features=1000)
# data_f = matrix.fit_transform(train_prim['text'],train_prim['Final Label'])
data_second_test = matrix.fit_transform(test_data_second_fe['text'],test_data_second_fe['final_label'])
print(data_second_test.shape)
print(sorted(matrix.vocabulary_))

(1500, 1000)
['abbott', 'academ', 'access', 'account', 'across', 'act', 'action', 'activ', 'ad', 'adult', 'affect', 'africa', 'agenc', 'agent', 'aid', 'air', 'airlin', 'airport', 'al', 'alabama', 'alberta', 'allow', 'amend', 'american', 'amid', 'amidst', 'amount', 'anim', 'announc', 'anti', 'appeal', 'appropri', 'approv', 'area', 'argentino', 'arizona', 'around', 'arriv', 'art', 'artist', 'arunach', 'asap', 'asian', 'ask', 'assembl', 'assess', 'assign', 'assist', 'associ', 'athlet', 'attack', 'austin', 'australia', 'australian', 'avail', 'avoid', 'awar', 'award', 'away', 'babi', 'back', 'bail', 'baker', 'ban', 'bank', 'base', 'basic', 'battl', 'bc', 'beach', 'beauti', 'beca', 'becom', 'behind', 'benefit', 'better', 'bill', 'board', 'boat', 'border', 'break', 'bring', 'british', 'build', 'busi', 'ca', 'california', 'call', 'camp', 'campus', 'canada', 'canadian', 'cancel', 'cancer', 'care', 'carolina', 'case', 'caus', 'center', 'centr', 'central', 'ceremoni', 'certif', 'chang', 'charg', 

In [54]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(data_fe_train.toarray(), y_train_f)

GaussianNB()

In [55]:
# predict class
y_pred_second = classifier.predict(data_second_test.toarray())

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm_second = confusion_matrix(test_data_second_fe['final_label'], y_pred_second)
cr_second = classification_report(test_data_second_fe['final_label'], y_pred_second)
accuracy_secondary = accuracy_score(test_data_second_fe['final_label'], y_pred_second)
print("Confusion Matrix\n",cm_second)
print("Classification Report\n", cr_second)
print("Accuracy\n", accuracy_secondary)

Confusion Matrix
 [[832 640]
 [ 10  18]]
Classification Report
               precision    recall  f1-score   support

       False       0.99      0.57      0.72      1472
        True       0.03      0.64      0.05        28

    accuracy                           0.57      1500
   macro avg       0.51      0.60      0.39      1500
weighted avg       0.97      0.57      0.71      1500

Accuracy
 0.5666666666666667


In [56]:
#Random Forest Classifier:
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(n_estimators = 100,max_depth = 5, random_state = 10)
classifier.fit(data_fe_train.toarray(), y_train_f)

RandomForestClassifier(max_depth=5, random_state=10)

In [57]:
# predict class
y_pred_second = classifier.predict(data_second_test.toarray())

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm_second = confusion_matrix(test_data_second_fe['final_label'], y_pred_second)
cr_second = classification_report(test_data_second_fe['final_label'], y_pred_second)
accuracy_secondary = accuracy_score(test_data_second_fe['final_label'], y_pred_second)
print("Confusion Matrix\n",cm_second)
print("Classification Report\n", cr_second)
print("Accuracy\n", accuracy_secondary)

Confusion Matrix
 [[787 685]
 [  7  21]]
Classification Report
               precision    recall  f1-score   support

       False       0.99      0.53      0.69      1472
        True       0.03      0.75      0.06        28

    accuracy                           0.54      1500
   macro avg       0.51      0.64      0.38      1500
weighted avg       0.97      0.54      0.68      1500

Accuracy
 0.5386666666666666


### Model Evaluation 2:

After applying Feature Engineering and adding new features in the dataset we built model using GaussianNB and Random Forest Classifier to see how accuracy is affected.

- In first case, where we have implemented Feature Engineering on Primary Dataset(NYT_VACCINATION), we have built two models using GaussianNB and Random Forest Classifier:
        - GaussianNB ~ 66% (An increase of 3% in accuracy from the model built without Feature Engineering)
        - RandomForestClassifier~ 81% (An increase of 18% in accuracy from the model built without Feature Engineering)
- In second case, where we have implemented Feature Engineering on Secondary Dataset(Change.org_Vaccination), we have built two models using GaussianNB and Random Forest Classifier:
        - GaussianNB ~ 57% (An increase of 12% in accuracy from the model built without Feature Engineering)
        - RandomForestClassifier~ 53%(An increase of 8% in accuracy from the model built without Feature Engineering)